# Reading the data into python

In [ ]:
# Supressing the warning messages
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Reading the dataset
import pandas as pd
import numpy as np


inventory_df = pd.read_csv(open(source_data))
inventory_df.sample(5)

In [ ]:
print('Shape before deleting duplicate values:', inventory_df.shape)

# Removing duplicate rows if any
inventory_df = inventory_df.drop_duplicates(['Brand Name','Product Name','Weight'])
print('Shape After deleting duplicate values:', inventory_df.shape)

# Printing sample data
inventory_df.sample(5)

# Defining the problem statement

#### Create a predictive model which gives relevant products recommendations

Predictors: Category, Sub Category, Brand Name, Product Rating, Name

# Basic Data Exploration

In [ ]:
# Looking at the sample rows in the data
inventory_df.sample(5)

In [ ]:
# Observing the summarized information of data
# Data types, Missing values based on number of non-null values Vs total rows etc.
# Remove those variables from data which have too many missing values (Missing Values > 30%)
# Remove Qualitative variables which cannot be used in Machine Learning
inventory_df.info()

In [ ]:
# Looking at the descriptive statistics of the data
inventory_df.describe(include='all')

In [ ]:
# Finding unique values for each column
# TO understand which column is categorical and which one is Continuous
# Typically if the numer of unique values are < 20 then the variable is likely to be a category otherwise continuous
inventory_df.nunique()

# Basic Data Exploration Results

Based on the basic exploration above, we can create a simple report of the data, noting down the observations regaring each column. Hence, creating a initial roadmap for further analysis.

The selected columns in this step are not final, further study will be done and then a final list will be created.

- Category     : Selected. Categorical.
- Sub Category : Selected. Categorical.
- Brand Name   : Selected. Categorical.
- Product Name: Selected. Categorical.
- Name         : Selected. Categorical.
- Weight       : Selected. Continuous.
- MRP/Unit     : Selected. Continuous.
- Qnty         : Selected. Continuous.
- Unit Qnty    : Selected. Continuous.
- Cost Price   : Selected. Continuous.
- Product ID   : Selected. Continuous.
- Product rating : Selected. Continuous.

# Visual Exploratory Data Analysis

Categorical variables: Bar plot

Continuous variables: Histogram

## Visualize distribution of all the Categorical Predictor variables in the data using bar plots
We can spot a categorical variable in the data by looking at the unique values in them. Typically a categorical variable contains less than 20 Unique values AND there is repetition of values, which means the data can be grouped by those unique values.

In [ ]:
# Plotting multiple bar charts at once for categorical variables
# Since there is no default function which can plot bar charts for multiple columns at once
# we are defining our own function for the same

def PlotBarCharts(inpData, colsToPlot):
    %matplotlib inline
    
    import matplotlib.pyplot as plt
    
    # Generating multiple subplots
    fig, subPlot=plt.subplots(nrows=1, ncols=len(colsToPlot), figsize=(50,5))
    fig.suptitle('Bar charts of: '+ str(colsToPlot))

    for colName, plotNumber in zip(colsToPlot, range(len(colsToPlot))):
        inpData.groupby(colName).size().plot(kind='bar',ax=subPlot[plotNumber])

In [ ]:
#####################################################################
# Calling the function for 5 columns
PlotBarCharts(inpData=inventory_df, 
              colsToPlot=['Category', 'Sub Category'])

In [ ]:
#####################################################################
# Calling the function for 5 columns
PlotBarCharts(inpData=inventory_df, 
              colsToPlot=['Simplified Category', 'Brand Name'])

# Bar Charts Interpretation
These bar charts represent the frequencies of each category in the Y-axis and the category names in the X-axis.

In this data, all the categorical columns except "Brand Name", "Name" and "Product Name" have satisfactory distribution for machine learning.

# Visualize distribution of all the Continuous Predictor variables in the data using histograms

In [ ]:
# Plotting histograms of multiple columns together

inventory_df.hist(['MRP/Unit', 'Rating '], figsize=(10, 5))

In [ ]:
inventory_df.columns

In [ ]:
# Plotting histograms of multiple columns together

inventory_df.hist(['Product ID '], figsize=(10,5))

# Histogram Interpretation
Histograms shows us the data distribution for a single continuous variable.

The ideal outcome for histogram is a bell curve or slightly skewed bell curve. If there is too much skewness, then outlier treatment should be done and the column should be re-examined, if that also does not solve the problem then only reject the column.

Selected Continuous Variables:

- MRP/Unit   : Selected. Slightly skewed distribution, acceptable.
- Qnty       : Selected. Slightly skewed distribution, acceptable.
- Unit Qnty  : Selected. Skewed distribution, not acceptable.
- Cost Price : Selected. Slightly skewed distribution, acceptable.


# Database Connection

In [ ]:
import sqlalchemy
import json
import os
notebook_path = os.getcwd()
con_path = os.path.join(notebook_path, "connection.json")
with open(con_path, "r") as read_file:
    con_j = json.load(read_file)
    username=con_j['username']
    password=con_j['password']
    database=con_j['database']
    server=con_j['server']
    port=con_j['port']
    engine = sqlalchemy.create_engine("mysql+pymysql://"+username+":"+password+"@"+server+":"+port+"/"+database)

# Outlier treatment
Outliers are extreme values in the data which are far away from most of the values. You can see them as the tails in the histogram.

Outlier must be treated one column at a time. As the treatment will be slightly different for each column.

Why I should treat the outliers?

Outliers bias the training of machine learning models. As the algorithm tries to fit the extreme value, it goes away from majority of the data.

There are below two options to treat outliers in the data.

- Option-1: Delete the outlier Records. Only if there are just few rows lost.
- Option-2: Impute the outlier values with a logical business value

In this data all the continuous variables have slightly skewed distribution, which is acceptable, hence no outlier treatment is required.




1. Rating <0 and >5 
2. simplfied category not in 'Groceries' ,'Fruits' ,'Dry Fruits'
3. Length of productid >12
4. MRP/Unit <0 and MRP/Unit>10000

In [ ]:
def outlier_treatment(datacolumn):
    sorted(datacolumn)
    Q1,Q3 = np.percentile(datacolumn , [25,75])
    IQR = Q3 - Q1
    lower_range = Q1 - (1.5 * IQR)
    upper_range = Q3 + (1.5 * IQR)
    return lower_range,upper_range

In [ ]:
lowerbound,upperbound = outlier_treatment(inventory_df['Rating '])

In [ ]:
# Records not within the boundary
inventory_df[(inventory_df['Rating '] < lowerbound) | (inventory_df['Rating '] > upperbound)]

In [ ]:
error_records_df = inventory_df[(inventory_df['Rating '] < lowerbound) | (inventory_df['Rating '] > upperbound)]
error_records_df.drop(['Sub Category'], axis=1, inplace=True)
error_records_df.rename(columns = {'Category':'Sub_Category_Name', 'Simplified Category':'Category_Name','Product Name':'Product_Name', 'MRP/Unit':'Price','Product ID ':'productid', 'Image URL':'Image', 'Rating ':'Product_Rating', 'Brand Name':'Brand'}, inplace= True)
error_records_df.to_csv(unprocessed_data, index=False)
error_records_df.to_sql('UnprocessedData', con=engine, if_exists='append', index=False)

In [ ]:
# simplfied category not in 'Groceries' ,'Fruits' ,'Dry Fruits'
inventory_df[(inventory_df['Simplified Category'] != 'Groceries') & (inventory_df['Simplified Category'] != 'Fruits') & (inventory_df['Simplified Category'] != 'Dry Fruits')]

In [ ]:
error_records_df =inventory_df[(inventory_df['Simplified Category'] != 'Groceries') & (inventory_df['Simplified Category'] != 'Fruits') & (inventory_df['Simplified Category'] != 'Dry Fruits')]
error_records_df.drop(['Sub Category'], axis=1, inplace=True)
error_records_df.rename(columns = {'Category':'Sub_Category_Name', 'Simplified Category':'Category_Name','Product Name':'Product_Name', 'MRP/Unit':'Price','Product ID ':'productid', 'Image URL':'Image', 'Rating ':'Product_Rating', 'Brand Name':'Brand'}, inplace= True)
error_records_df.to_csv(unprocessed_data, mode='a', index=False, header=False)
error_records_df.to_sql('UnprocessedData', con=engine, if_exists='append', index=False)

In [ ]:
# Length of productid > 12
inventory_df[inventory_df['Product ID '].astype(str).str.len() > 12]

In [ ]:
error_records_df = inventory_df[inventory_df['Product ID '].astype(str).str.len() > 12]
error_records_df.drop(['Sub Category'], axis=1, inplace=True)
error_records_df.rename(columns = {'Category':'Sub_Category_Name', 'Simplified Category':'Category_Name','Product Name':'Product_Name', 'MRP/Unit':'Price','Product ID ':'productid', 'Image URL':'Image', 'Rating ':'Product_Rating', 'Brand Name':'Brand'}, inplace= True)
error_records_df.to_csv(unprocessed_data, mode='a', index=False, header=False)
error_records_df.to_sql('UnprocessedData', con=engine, if_exists='append', index=False)

In [ ]:
# MRP/Unit <0 and MRP/Unit>10000
inventory_df[(inventory_df['MRP/Unit'] < 0) | (inventory_df['MRP/Unit'] > 10000)]

In [ ]:
error_records_df = inventory_df[(inventory_df['MRP/Unit'] < 0) | (inventory_df['MRP/Unit'] > 10000)]
error_records_df.drop(['Sub Category'], axis=1, inplace=True)
error_records_df.rename(columns = {'Category':'Sub_Category_Name', 'Simplified Category':'Category_Name','Product Name':'Product_Name', 'MRP/Unit':'Price','Product ID ':'productid', 'Image URL':'Image', 'Rating ':'Product_Rating', 'Brand Name':'Brand'}, inplace= True)
error_records_df.to_csv(unprocessed_data, mode='a', index=False, header=False)
error_records_df.to_sql('UnprocessedData', con=engine, if_exists='append', index=False)

In [ ]:
# remove the outliers from the dataset

inventory_df.drop(inventory_df[ (inventory_df['Rating '] > upperbound) | (inventory_df['Rating '] < lowerbound) ].index , inplace=True)

inventory_df.drop(inventory_df[(inventory_df['Simplified Category'] != 'Groceries') & (inventory_df['Simplified Category'] != 'Fruits') & (inventory_df['Simplified Category'] != 'Dry Fruits')].index, inplace=True)

inventory_df.drop(inventory_df[inventory_df['Product ID '].astype(str).str.len() > 12].index, inplace=True)

inventory_df.drop(inventory_df[(inventory_df['MRP/Unit'] < 0) | (inventory_df['MRP/Unit'] > 10000)].index, inplace=True)


# Missing values treatment
Missing values are treated for each column separately.

If a column has more than 30% data missing, then missing value treatment cannot be done. That column must be rejected because too much information is missing.

There are below options for treating missing values in data.

- Delete the missing value rows if there are only few records
- Impute the missing values with MEDIAN value for continuous variables
- Impute the missing values with MODE value for categorical variables
- Interpolate the values based on nearby values
- Interpolate the values based on business logic

In [ ]:
# Finding how many missing values are there for each column
inventory_df.isnull().sum()

In [ ]:
error_records_df = inventory_df[inventory_df.isnull().any(axis=1)]
error_records_df.drop(['Sub Category'], axis=1, inplace=True)
error_records_df.rename(columns = {'Category':'Sub_Category_Name', 'Simplified Category':'Category_Name','Product Name':'Product_Name', 'MRP/Unit':'Price','Product ID ':'productid', 'Image URL':'Image', 'Rating ':'Product_Rating', 'Brand Name':'Brand'}, inplace= True)
error_records_df.to_csv(unprocessed_data, mode='a', index=False, header=False)
error_records_df.to_sql('UnprocessedData', con=engine, if_exists='append', index=False)
error_records_df

In [ ]:
inventory_df.replace('', np.nan, inplace=True)

In [ ]:
inventory_df.dropna(inplace=True)

# Finding how many missing values are there for each column
inventory_df.isnull().sum()

All the missing values are removed now.

# Data Cleansing

In [ ]:
# Removing the leading and trailing spaces of columns
inventory_df.columns = inventory_df.columns.str.strip()

In [ ]:
inventory_df.columns

# Find Correalation between attributes

In [ ]:
# To find the correlation among
# the columns using pearson method
inventory_df.corr(method ='pearson')

# Save the file into local machine

In [ ]:
inventory_df.drop(['Sub Category'], axis=1, inplace=True)

In [ ]:
inventory_df.rename(columns = {'Category':'Sub_Category_Name', 'Simplified Category':'Category_Name','Product Name':'Product_Name', 'MRP/Unit':'Price','Product ID':'productid', 'Image URL':'Image', 'Rating':'Product_Rating', 'Brand Name':'Brand'}, inplace= True)

In [ ]:
inventory_df.to_csv(preprocessed_data, index=False)

# Save the data into local database

In [ ]:
inventory_df.to_sql('PreprocessedData', con=engine, if_exists='replace', index=False)